In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from googleapiclient.discovery import build


import httpx

class CustomHTTPClient(httpx.Client):
    def __init__(self, *args, **kwargs):
        kwargs.pop("proxies", None) # Remove the 'proxies' argument if present
        super().__init__(*args, **kwargs)

load_dotenv()
client = OpenAI(api_key=os.environ['API_KEY'], http_client=CustomHTTPClient())
model_name = "gpt-3.5-turbo"

In [4]:
import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

In [5]:
data_path = "training.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 10
First example:
{'role': 'user', 'content': '폭삭 속았수다'}
{'role': 'assistant', 'content': '정말 수고많으셨습니다'}


In [6]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [7]:
# 学習データのアップロード
client.files.create(
  file=open("training.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-Bj6Qs28Dgx7QM4ebT5mjSa', bytes=1125, created_at=1744258935, filename='training.jsonl', object='file', purpose='fine-tune', status='processed', expires_at=None, status_details=None)

In [ ]:
# ファインチューニングジョブの開始
client.fine_tuning.jobs.create(
  training_file = "file-Bj6Qs28Dgx7QM4ebT5mjSa", 
  model = model_name
)

In [8]:
client.fine_tuning.jobs.list(limit=10)

SyncCursorPage[FineTuningJob](data=[], has_more=False, object='list')